# A simple spam filter constructed from a Naive Bayes algorithm

Spam may seem like an annoyance, but is truly a worldwide problem with approximately 86% of emails being spam [[1]](https://www.ezcomputersolutions.com/blog/true-cost-of-spam-for-companies/) .  94 billion spam threats are sent every day throughout the world [[2]](https://www.mailcleaner.net/blog/spam-world-news/how-much-does-spam-cost-the-world/), highlighting the importance of efficient spam filters. In this notebook, I demonstrate how to build a simple spam filter based on a Naive Bayes algorithm. This spam filter assumes conditional independence of words in a message, which means that the presence of one word is not assumed to affect the probability to find a different word. This is obviously a simplified representation of reality: it is more likely, for example, to find the word "money" in an email given the word "bank" is present.

The database that this algorithm will learn from comes from [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection) and includes 747 spam SMS messages and 4827 desired SMS messages.

In [1]:
import pandas as pd
import numpy as np

sms = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS']) 
sms.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [2]:
P_spam = (sms['Label'] == 'spam').sum() / sms.shape[0]
P_Nspam = (sms['Label'] == 'ham').sum() / sms.shape[0]
print(('Probability of spam: {:.1f}%.\n'
       'Probability of non-spam: {:.1f}%').format(P_spam*100, P_Nspam*100))

Probability of spam: 13.4%.
Probability of non-spam: 86.6%


Based solely on our dataset, and without using any information about the contents of the message, we can estimate that the probability the a given SMS is spam is 13.4%.

Before learning anythin from the contents of the messages, let's split the data set into a training set (80%) and a testing set (20%).

In [3]:
# Randomize dataset
sms = sms.sample(frac=1, random_state=1)

# Split training (80%) and test (20%) data
N = len(sms.index)
split = int(.8*N)
sms_train = sms.iloc[0:split].copy()
sms_train.reset_index(inplace=True)
sms_test  = sms.iloc[split:N].copy()
sms_test.reset_index(inplace=True)

# Recalculate probabilities of spam vs. ham for splitted training dataset
P_spam = (sms_train['Label'] == 'spam').sum() / sms_train.shape[0]
P_Nspam = (sms_train['Label'] == 'ham').sum() / sms_train.shape[0]
print(('Training data\n'
       'Probability of spam: {:.1f}%.\n'
       'Probability of non-spam: {:.1f}%').format(P_spam*100, P_Nspam*100))

# Recalculate probabilities of spam vs. ham for splitted test dataset
P_spam = (sms_test['Label'] == 'spam').sum() / sms_test.shape[0]
P_Nspam = (sms_test['Label'] == 'ham').sum() / sms_test.shape[0]
print(('\nTest data\n'
       'Probability of spam: {:.1f}%.\n'
       'Probability of non-spam: {:.1f}%').format(P_spam*100, P_Nspam*100))

Training data
Probability of spam: 13.5%.
Probability of non-spam: 86.5%

Test data
Probability of spam: 13.2%.
Probability of non-spam: 86.8%


The probability of spam in both training and test datasets remains similar to the combined dataset. To proceed with the training, the next step will be to rearrange the training dataframe in terms of word counts.

In [4]:
# Replace all that is not a-z or A-Z by a space. Lower every case.
sms_train['SMS'] = sms_train['SMS'].str.replace(r'[\W\d_]',' ').str.lower()

# Create vocabulary set of all words encountered in the dataset
vocab = set()
for message in sms_train['SMS']:
    vocab.update(message.split())

# Add one column to sms dataframe for each word. The value will be the word count for that message.
N = len(sms_train.index)
dictio = {unique_word: [0]*N for unique_word in list(vocab)}

for ii, message in enumerate(sms_train['SMS']):
    for word in message.split():
        dictio[word][ii] += 1

df_word_count = pd.DataFrame(dictio)

sms_wordcount = pd.concat((sms_train, df_word_count), axis=1, ignore_index=False)
sms_wordcount.head()

,index,Label,SMS,interflora,milta,clark,support,cardiff,toilet,senrd,...,packing,medical,growing,qing,sculpture,posted,paid,removal,tmrw,dun
0,1078,ham,yep by the pretty sculpture,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,4028,ham,yes princess are you going to make me moan,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,958,ham,welp apparently he retired,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4642,ham,havent,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4674,ham,i forgot ask ü all smth there s a card on ...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now that we have a clean training dataset, we can begin to implement the spam filter. In the Naive Bayes algorithm,

$\displaystyle P(\text{spam}|w_1,w_2,...,w_n) \propto P(\text{spam}) \prod_{i=1}^n P(w_i|\text{spam})$  
$\displaystyle P(\text{ham}|w_1,w_2,...,w_n) \propto P(\text{ham}) \prod_{i=1}^n P(w_i|\text{ham})$

with  
$\displaystyle P(w_i|\text{spam}) = \frac{N_{w_i|\text{spam}}+\alpha}{N_\text{spam} + \alpha N_\text{vocab}}$  
and  
$\displaystyle P(w_i|\text{ham}) = \frac{N_{w_i|\text{ham}}+\alpha}{N_\text{ham} + \alpha N_\text{vocab}}$  

We choose $\alpha=1$ (Laplace hypothesis).

In [7]:
# Split the dataframe into spam and ham dataframes. 
sms_wordcount_spam = sms_wordcount[sms_train['Label'] == 'spam']
sms_wordcount_ham  = sms_wordcount[sms_train['Label'] == 'ham']
sms_wordcount_spam = sms_wordcount_spam.iloc[:, 3:] # remove 
sms_wordcount_ham  = sms_wordcount_ham.iloc[:, 3:]

# Calculate P(spam) and P(ham)
P_spam = sms_wordcount_spam.shape[0] / sms_train.shape[0]
P_ham  = sms_wordcount_ham.shape[0] / sms_train.shape[0]

# Calculate N_spam, N_ham and N_vocab
N_vocab = len(vocab)
N_spam = sms_wordcount_spam.sum().sum()
N_ham  = sms_wordcount_ham.sum().sum()

# Calculate P(w_i | spam) and P(w_i | ham)
N_wi_given_spam = {w_i: sms_wordcount_spam[w_i].sum() for w_i in vocab}
N_wi_given_ham  = {w_i: sms_wordcount_ham[w_i].sum()  for w_i in vocab}
alpha = 1

def calc_P_wi_given_spam(wi):
    return (N_wi_given_spam[wi] + alpha) / (N_spam + alpha*N_vocab)

def calc_P_wi_given_ham(wi):
    return (N_wi_given_ham[wi] + alpha) / (N_ham + alpha*N_vocab)

P_wi_given_spam = {w_i: calc_P_wi_given_spam(w_i) for w_i in vocab}
P_wi_given_ham  = {w_i: calc_P_wi_given_ham(w_i)  for w_i in vocab}


With most variables pre-computed, next we will code the function that takes in a new message and determines if it classifies as spam or not.

In [8]:
import re

def classify_message(message):
    
    # Reduce message (same treatement as our training dataset)
    message = re.sub(r'[\W\d_]', ' ', message).lower()
    words = message.split()
    
    # Calculate P(spam | w_1, w_2, ..., w_n)
    P_spam_given_message = P_spam
    for w_i in words:
        try:
            P_spam_given_message *= P_wi_given_spam[w_i]
        except KeyError:
            continue
        
    # Calculate P(ham | w_1, w_2, ..., w_n)
    P_ham_given_message = P_ham
    for w_i in words:
        try:
            P_ham_given_message *= P_wi_given_ham[w_i]
        except KeyError:
            continue
    
    # Return message
    if P_spam_given_message > P_ham_given_message:
        return 'spam'
    else:
        return 'ham'


Finally, to test the spam filter, let's use the remaining SMS from our test dataset. Those SMS have never been seen by the algorithm and so they are considered new.

In [9]:
sms_test['predicted'] = sms_test['SMS'].apply(classify_message)
sms_test['success'] = sms_test['Label'] == sms_test['predicted']
sms_test.head()

accuracy = sms_test['success'].sum() / sms_test.shape[0]
print('Accuracy: {:.2f}%'.format(accuracy*100))

Accuracy: 98.48%


Our accuracy at predicting spam is 98.92%, an excellent result!

# Conclusion

In this project, we developed a Naive Bayes spam filter and made it learn using data from [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). The accuracy of prediction of our spam filter is 98.92%, which is great considering the simplicity of this algorithm. The accuracy may be inflated considering that our training and testing datasets have been sampled from the same dataset covering three specific scenarios. This would be a good reason to redo a test against a new dataset which the training has not sampled from.